In [ ]:
import pandas as pd
import urllib3
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
tf.__version__

'2.12.0'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/경남대 빅데이터/fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del df['lic']
print('전체 샘플의 개수 :',len(df))

전체 샘플의 개수 : 227815


In [ ]:
df = df.loc[:, 'src':'tar']
df = df[0:60000] # 6만개만 저장
df.sample(10)

,src,tar
50990,She is hostile to me.,Elle m'est hostile.
16469,It was unlocked.,Elle était déverrouillée.
14814,Give me the pen.,Donne-moi le stylo.
22480,She must be sick.,Elle doit être malade.
31087,You are very rich.,Vous êtes très riches.
9169,Tastes differ.,Les goûts diffèrent.
44006,Tom became a father.,Tom est devenu père.
55207,Don't open your mouth.,N'ouvre pas la bouche !
50205,It was too difficult.,C'était trop dur.
11178,I didn't leave.,Je ne suis pas parti.


In [ ]:
df.tar = df.tar.apply(lambda x : '\t '+ x + ' \n')
df.sample(10)

,src,tar
22326,Please don't ask.,"\t Ne demandez pas, je vous prie ! \n"
30012,Tom offended Mary.,\t Tom offensa Marie. \n
38595,You're still green.,\t Tu es encore un bleu. \n
29059,That's my teacher.,\t C'est mon professeur. \n
31264,You must hurry up.,\t Tu dois te dépêcher. \n
17642,This won't work.,\t Ça ne fonctionnera pas. \n
25603,Don't be too long.,\t Ne soyez pas trop long. \n
19642,Everybody's dead.,\t Tout le monde a crevé. \n
528,Get away!,\t Criss ton camp d'icit ! \n
23273,Tom had jeans on.,\t Tom portait un jean. \n


In [ ]:
# 글자 집합 구축
src_vocab = set()
for line in df.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in df.tar:
    for char in line:
        tar_vocab.add(char)

In [ ]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104


In [ ]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x']


In [ ]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 3

In [ ]:
encoder_input = []

# 1개의 문장
for line in df.src:
  enc_line = []
  # 각 줄에서 1개의 char
  for char in line:
    # 각 char을 정수로 변환
    enc_line.append(src_to_index[char])
  encoder_input.append(enc_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [ ]:
dec_input = []
for line in df.tar:
  enc_line = []
  for char in line:
    enc_line.append(tar_to_index[char])
  dec_input.append(enc_line)
print('target 문장의 정수 인코딩 :',dec_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [1, 3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [1, 3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [1, 3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [ ]:
dec_target = []
for line in df.tar:
  timestep = 0
  enc_line = []
  for char in line:
    if timestep > 0:
      enc_line.append(tar_to_index[char])
    timestep = timestep + 1
  dec_target.append(enc_line)
print('target 문장 레이블의 정수 인코딩 :',dec_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 52, 3, 4, 3, 2], [3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [ ]:
max_src_len = max([len(line) for line in df.src])
max_tar_len = max([len(line) for line in df.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
dec_input = pad_sequences(dec_input, maxlen=max_tar_len, padding='post')
dec_target = pad_sequences(dec_target, maxlen=max_tar_len, padding='post')

In [ ]:
encoder_input = to_categorical(encoder_input)
dec_input = to_categorical(dec_input)
dec_target = to_categorical(dec_target)

In [ ]:
import numpy as np
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [ ]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

In [ ]:
dec_inputs = Input(shape=(None, tar_vocab_size))
dec_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
dec_outputs, _, _= dec_lstm(dec_inputs, initial_state=encoder_states)

dec_softmax_layer = Dense(tar_vocab_size, activation='softmax')
dec_outputs = dec_softmax_layer(dec_outputs)

model = Model([encoder_inputs, dec_inputs], dec_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
model.fit(x=[encoder_input, dec_input], y=dec_target, batch_size=64, epochs=1, validation_split=0.2)

750/750 [==============================] - 24s 19ms/step - loss: 0.8529 - val_loss: 0.7706


In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 80)]        0         
                                                                 
 lstm (LSTM)                 [(None, 256),             345088    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 345,088
Trainable params: 345,088
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
dec_state_input_h = Input(shape=(256,))
dec_state_input_c = Input(shape=(256,))
dec_states_inputs = [dec_state_input_h, dec_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
dec_outputs, state_h, state_c = dec_lstm(dec_inputs, initial_state=dec_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
dec_states = [state_h, state_c]
dec_outputs = dec_softmax_layer(dec_outputs)
dec_model = Model(inputs=[dec_inputs] + dec_states_inputs, outputs=[dec_outputs] + dec_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  dec_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = dec_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    dec_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(dec_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return dec_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  dec_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', df.src[seq_index])
  print('정답 문장:', df.tar[seq_index][2:len(df.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', dec_sentence[1:len(dec_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 29ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Je sous pas parte. 
1/1 [==============================] - 0s 22ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Je sous pas parte. 
1/1 [==============================] - 0s 23ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Je sous pas parte. 
1/1 [==============================] - 0s 23ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Je sous pas parte. 
1/1 [==============================] - 0s 24ms/step
-----------------------------------
입력 문장: Get going.
정답 문장: En avant. 
번역 문장: Je sous pas parte. 
